# HW 4
## Tim Demetriades
### AAI 695 WS2 - Shucheng Yu
4/6/2022

#### Question 5 - Programming
#### In this programming problem, you will get familiar with building a neural network using backpropagation. You are supposed to implement the following steps: 
#### Step 1: Use our “titanic” dataset in homework #3, and split data in the same way you did in homework #3 – 80% as training and 20% test sets; 

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import classification_report, confusion_matrix
from tabulate import tabulate

In [2]:
titanic_df = pd.read_csv("Titanic.csv")

In [3]:
titanic_df.head()

,Unnamed: 0,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1st,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.337494,B5,Southampton,2,NaN,"St Louis, MO"
1,2,1st,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.550003,C22 C26,Southampton,11,NaN,"Montreal, PQ / Chesterville, ON"
2,3,1st,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,4,1st,0,"Allison, Mr. Hudson Joshua Crei",male,30.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,5,1st,0,"Allison, Mrs. Hudson J C (Bessi",female,25.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [4]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1309 non-null   int64  
 1   pclass      1309 non-null   object 
 2   survived    1309 non-null   int64  
 3   name        1309 non-null   object 
 4   sex         1309 non-null   object 
 5   age         1046 non-null   float64
 6   sibsp       1309 non-null   int64  
 7   parch       1309 non-null   int64  
 8   ticket      1309 non-null   object 
 9   fare        1308 non-null   float64
 10  cabin       295 non-null    object 
 11  embarked    1307 non-null   object 
 12  boat        486 non-null    object 
 13  body        121 non-null    float64
 14  home.dest   745 non-null    object 
dtypes: float64(3), int64(4), object(8)
memory usage: 153.5+ KB


In [5]:
titanic_df.describe()

,Unnamed: 0,survived,age,sibsp,parch,fare,body
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,121.000000
mean,655.000000,0.381971,29.881135,0.498854,0.385027,33.295479,160.809917
std,378.020061,0.486055,14.413500,1.041658,0.865560,51.758669,97.696922
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000,1.000000
25%,328.000000,0.000000,21.000000,0.000000,0.000000,7.895800,72.000000
50%,655.000000,0.000000,28.000000,0.000000,0.000000,14.454200,155.000000
75%,982.000000,1.000000,39.000000,1.000000,0.000000,31.275000,256.000000
max,1309.000000,1.000000,80.000000,8.000000,9.000000,512.329224,328.000000


Here we will turn the age and fare from categorical to numerical.

In [6]:
average_age = titanic_df.age.mean()
average_fare = titanic_df.fare.mean()

In [7]:
titanic_df.age.fillna(average_age, inplace = True)
titanic_df.fare.fillna(average_fare, inplace = True)

Here we replace these strings for the pclass column with integers.

In [8]:
titanic_df.pclass.unique()    # show all ticket classes

array(['1st', '2nd', '3rd'], dtype=object)

In [9]:
pclass_dict = {'1st': 1, '2nd': 2, '3rd': 3}

In [10]:
titanic_df.pclass.replace(pclass_dict, inplace = True)    # replace strings with ints

Finally we replace male with 0 and female with 1 and then split the set into a training set and a test set.

In [11]:
sex_dict = {'male': 0, 'female': 1}    # male = 0, female = 1

In [12]:
titanic_df.sex.replace(sex_dict, inplace = True)

Here is where the full dataset is split into a training and test set.

In [13]:
train_df, test_df = train_test_split(titanic_df,  test_size = 0.2, random_state = 42)

#### Step 2: Fit a neural network using independent variables ‘pclass + sex + age + sibsp’ and  dependent variable ‘survived’. Fill in n/a attributes with the average of the same attributes from other training examples. Use 2 hidden layers and set the activation functions for both the hidden and output layer to be the sigmoid function. Set “solver” parameter as either SGD (stochastic gradient descend) or Adam (similar to SGD but optimized performance with mini batches). You can adjust parameter “alpha” for regularization (to control overfitting) and other parameters such as “learning rate” and “momentum” as needed.   

First the features and labels are split for the training and test data.

In [14]:
X_train = train_df[['pclass', 'sex', 'age', 'sibsp']]
X_test = test_df[['pclass', 'sex', 'age', 'sibsp']]
y_train = train_df['survived']
y_test = test_df['survived']

In [15]:
X_train

,pclass,sex,age,sibsp
772,3,0,17.000000,0
543,2,0,36.000000,0
289,1,1,18.000000,0
10,1,0,47.000000,1
147,1,0,29.881135,0
...,...,...,...,...
1095,3,1,29.881135,0
1130,3,1,18.000000,0
1294,3,0,28.500000,0
860,3,1,26.000000,0


In [16]:
X_test

,pclass,sex,age,sibsp
1148,3,0,35.000000,0
1049,3,0,20.000000,1
982,3,0,29.881135,0
808,3,0,29.881135,0
1195,3,0,29.881135,0
...,...,...,...,...
572,2,1,28.000000,0
140,1,0,23.000000,0
1182,3,1,21.000000,0
312,1,0,50.000000,1


Since Multi-layer Perceptron is sensitive to feature scaling, we should standardize features by removing the mean and scaling to unit variance.

Formula: `z = (x - u) / s`

In [17]:
scaler = StandardScaler()
scaler.fit(X_train)    # only fit to training data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)  

In [18]:
X_train[0]

array([ 0.84035898, -0.73927586, -0.98672562, -0.4959643 ])

In [19]:
X_test[0]

array([ 0.84035898, -0.73927586,  0.42270133, -0.4959643 ])

Here a MLP Classifier is created and fit to the training data.

It uses the **adam solver**, a **learning rate of 0.0001**, the **logistic (sigmoid) activation function**, and **2 hidden layers with 4 neurons each**. These hyperparameters were chosen based on some minor testing to see which gave the best performance.

In [20]:
classifier = MLPClassifier(solver = 'adam', 
                           alpha = 0.0001,
                           activation = 'logistic',    # sigmoid
                           random_state = 42,
                           hidden_layer_sizes = (4, 4))    # two hidden layers with 4 neurons each
classifier.fit(X_train, y_train)

C:\Users\timjr\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(activation='logistic', hidden_layer_sizes=(4, 4), random_state=42)

Here we can use the `.score()` method to see the mean accuracy on the training and test data to get a general idea of how well the model performs.

In [21]:
classifier.score(X_train, y_train)    # mean accuracy

0.8061127029608405

In [22]:
classifier.score(X_test, y_test)    # mean accuracy

0.7709923664122137

#### Step 3: Check the performance of the model with out-of-sample accuracy, defined as:
#### - out-of-sample percent survivors correctly predicted (on test set)  
#### - out-of-sample percent fatalities correctly predicted (on test set)
#### Please try two different network structures (i.e., number of neurons at each hidden layer) and show their respective accuracy.  

Below the predictions for both the training and test data are obtained.

In [23]:
predict_train = classifier.predict(X_train)
predict_test = classifier.predict(X_test)

Using the predictions a confusion matrix and classification report can be created. The confusion matrix is then used to calculate the accuracy for both the number of survivors and number of fatalities correctly predicted.

In [24]:
tn_train, fp_train, fn_train, tp_train = confusion_matrix(y_train, predict_train).ravel()
print(confusion_matrix(y_train, predict_train))
print(classification_report(y_train, predict_train))

[[595  70]
 [133 249]]
              precision    recall  f1-score   support

           0       0.82      0.89      0.85       665
           1       0.78      0.65      0.71       382

    accuracy                           0.81      1047
   macro avg       0.80      0.77      0.78      1047
weighted avg       0.80      0.81      0.80      1047



In [25]:
tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, predict_test).ravel()
print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test))

[[129  15]
 [ 45  73]]
              precision    recall  f1-score   support

           0       0.74      0.90      0.81       144
           1       0.83      0.62      0.71       118

    accuracy                           0.77       262
   macro avg       0.79      0.76      0.76       262
weighted avg       0.78      0.77      0.77       262



In [26]:
survivors_accuracy = tp_test / (tp_test + fn_test)
print(f'Percent Survivors Correctly Predicted: {survivors_accuracy:.2%}')

Percent Survivors Correctly Predicted: 61.86%


In [27]:
fatalities_accuracy = tn_test / (tn_test + fp_test)
print(f'Percent Fatalities Correctly Predicted: {fatalities_accuracy:.2%}')

Percent Fatalities Correctly Predicted: 89.58%


As can be seen above, this network correctly predicts around **62% of survivors and around 90% of fatalities** (which match the recall values for the test data classification report).

While the fatality accuracy is pretty high, the survivor accuracy is still pretty low. Let's see if we can improve this with a different network structure.

First let's try to **increase the number of neurons in both of the two hidden layers to 8**.

In [28]:
classifier_2 = MLPClassifier(solver = 'adam', 
                             alpha = 0.0001,
                             activation = 'logistic',    # sigmoid
                             random_state = 42,
                             hidden_layer_sizes = (8, 8))    # two hidden layers with 8 neurons each
classifier_2.fit(X_train, y_train)

C:\Users\timjr\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(activation='logistic', hidden_layer_sizes=(8, 8), random_state=42)

In [29]:
classifier_2.score(X_train, y_train)    # mean accuracy

0.8089780324737345

In [30]:
classifier_2.score(X_test, y_test)    # mean accuracy

0.7442748091603053

In [31]:
predict_train = classifier_2.predict(X_train)
predict_test = classifier_2.predict(X_test)
tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, predict_test).ravel()

In [32]:
survivors_accuracy = tp_test / (tp_test + fn_test)
print(f'Percent Survivors Correctly Predicted: {survivors_accuracy:.2%}')

Percent Survivors Correctly Predicted: 51.69%


In [33]:
fatalities_accuracy = tn_test / (tn_test + fp_test)
print(f'Percent Fatalities Correctly Predicted: {fatalities_accuracy:.2%}')

Percent Fatalities Correctly Predicted: 93.06%


**The survivor accuracy has decreased dramatically to around 52% while the fatality accuracy has increased to around 93%**.

Let's try one more network structure. This time, **we will bring the number of neurons down from 8 to 6** (still higher than the original number of neurons of 4).

In [34]:
classifier_3 = MLPClassifier(solver = 'adam', 
                             alpha = 0.0001,
                             activation = 'logistic',    # sigmoid
                             random_state = 42,
                             hidden_layer_sizes = (6, 6))    # two hidden layers with 4 neurons each
classifier_3.fit(X_train, y_train)

C:\Users\timjr\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(activation='logistic', hidden_layer_sizes=(6, 6), random_state=42)

In [35]:
classifier_3.score(X_train, y_train)    # mean accuracy

0.8042024832855779

In [36]:
classifier_3.score(X_test, y_test)    # mean accuracy

0.7748091603053435

In [37]:
predict_train = classifier_3.predict(X_train)
predict_test = classifier_3.predict(X_test)
tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, predict_test).ravel()

In [38]:
survivors_accuracy = tp_test / (tp_test + fn_test)
print(f'Percent Survivors Correctly Predicted: {survivors_accuracy:.2%}')

Percent Survivors Correctly Predicted: 62.71%


In [39]:
fatalities_accuracy = tn_test / (tn_test + fp_test)
print(f'Percent Fatalities Correctly Predicted: {fatalities_accuracy:.2%}')

Percent Fatalities Correctly Predicted: 89.58%


Here we see that the **survivor accuracy is now around 63%**, which is slightly higher than when using 4 neurons in the two hidden layers, and the **fatality accuracy is around 90%**, the same as with 4 neurons.

This network seems to give the best results. Of note, the weight optimizer (solver) used is **adam**, which gave better results than **sgd** (not shown). 

#### Step 4: Compare the out-of-sample accuracy (as defined in step 3) with the random forest obtained in homework #3. (You can either use a table or plot the results of the two algorithms in one figure). Explain any difference in accuracy. 

In [40]:
# From Random Forest in HW2
rf_survivors_accuracy = 0.5000
rf_fatalities_accuracy = 0.9583

In [59]:
table = [['Neural Network'],
         ['Survivor Accuracy', 'Fatality Accuracy'],
         [f'{survivors_accuracy:.2%}', f'{fatalities_accuracy:.2%}'],
         [''],
         ['Random Forest'],
         ['Survivor Accuracy', 'Fatality Accuracy'],
         [f'{rf_survivors_accuracy:.2%}', f'{rf_fatalities_accuracy:.2%}']]

In [60]:
print(tabulate(table))

-----------------  -----------------
Neural Network
Survivor Accuracy  Fatality Accuracy
62.71%             89.58%

Random Forest
Survivor Accuracy  Fatality Accuracy
50.00%             95.83%
-----------------  -----------------


The above table shows the survivor and fatality accuracy for both the Random Forest model created for HW3 and the Neural Network model created for this HW.

As can be seen, while the fatality accuracy has gone down by around 6%, the survivor accuracy has increased by over 12% for the Neural Network. As a result we can say that this model performed better than the Random Forest model.

It is not always clear from the start which model will perform better for a given problem, which is why it is important to try different models and compare the results. This is essentially what was done here, with the Random Forest and Neural Network models both producing different results. From what we found, the Neural Network model seems to perform slightly better.

Generally, Neural Networks are more complex than Random Forests and therefore if they produce similar results than it will be better to just go with a Random Forest. Random Forests can do a great job at classifying binary data, which is what we have here. Neural Networks can also work for binary classification, but tend to need more data. Also, with Neural Networks, it can be difficult to explain why they make the decisions they make ('black box') compared to with Random Forests.

However, as seen here, the Neural Network performed better than the Random Forest given the dataset, particularly with the configured hyperparameters (solver, numebr of hidden layers, number of neurons).